In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Загрузка данных
df = pd.read_csv('games.csv', sep=',')

# Убедимся, что данные загружены корректно
df.head()


,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [8]:
#Преобразование текстовых данных в числовой формат
df['victory_status'] = df['victory_status'].astype('category').cat.codes
df['winner'] = df['winner'].astype('category').cat.codes
df['opening_eco'] = df['opening_eco'].astype('category').cat.codes
df['opening_name'] = df['opening_name'].astype('category').cat.codes

# Выберем признаки для обучения
features = df[['rated', 'turns', 'victory_status', 'white_rating', 'black_rating', 'opening_eco', 'opening_ply']]
target = df['winner']



In [9]:
# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Нормализация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Создание модели
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # 1 выход, так как это бинарная классификация (0 или 1)
])

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Оценка производительности модели
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')



Epoch 1/10


502/502 [==============================] - 2s 2ms/step - loss: -16.6899 - accuracy: 0.0718 - val_loss: -83.4948 - val_accuracy: 0.0850
Epoch 2/10
502/502 [==============================] - 1s 2ms/step - loss: -363.6432 - accuracy: 0.1054 - val_loss: -876.0141 - val_accuracy: 0.1119
Epoch 3/10
502/502 [==============================] - 1s 2ms/step - loss: -1687.4061 - accuracy: 0.1192 - val_loss: -2993.3215 - val_accuracy: 0.1174
Epoch 4/10
502/502 [==============================] - 1s 2ms/step - loss: -4391.0532 - accuracy: 0.1234 - val_loss: -6663.9087 - val_accuracy: 0.1002
Epoch 5/10
502/502 [==============================] - 1s 2ms/step - loss: -8654.6211 - accuracy: 0.1165 - val_loss: -12052.1289 - val_accuracy: 0.1199
Epoch 6/10
502/502 [==============================] - 1s 2ms/step - loss: -14665.9775 - accuracy: 0.1174 - val_loss: -19460.0312 - val_accuracy: 0.1209
Epoch 7/10
502/502 [==============================] - 1s 2ms/step - loss: -22640.4297 - accuracy: 0.

In [10]:
# Сохранение модели
model.save('chess_model.h5')

# Загрузка модели
loaded_model = tf.keras.models.load_model('chess_model.h5')

G:\Python Anaconda\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tkinter import Tk
from tkinter import Label, Button, OptionMenu, StringVar, Entry
from tkinter import END

def suggest_best_move(opponent_move):
    # Предобработка данных хода противника
    opponent_move_data = preprocess_opponent_move(opponent_move)
    
    # Предсказание с использованием загруженной модели
    prediction = loaded_model.predict(opponent_move_data)
    
    # Отображение рекомендуемого хода
    best_move = postprocess_best_move(prediction)
    return best_move

# ...

def preprocess_opponent_move(opponent_move):
    # Преобразование входных данных в формат, понимаемый моделью
    # Ваши шаги по предобработке данных хода противника здесь
    
    # Пример: Просто преобразуем ход противника в список значений
    processed_data = [int(opponent_move)]
    
    return processed_data

# ...


def suggest_best_move(opponent_move):
    # Предобработка данных хода противника
    opponent_move_data = preprocess_opponent_move(opponent_move)
    
    # Предсказание с использованием загруженной модели
    prediction = loaded_model.predict(opponent_move_data)
    
    # Отображение рекомендуемого хода
    best_move = postprocess_best_move(prediction)
    return best_move

def preprocess_opponent_move(opponent_move):
    # Преобразование входных данных в формат, понимаемый моделью
    # Ваши шаги по предобработке данных хода противника здесь
    
    # Пример: Просто преобразуем ход противника в список значений
    processed_data = [int(opponent_move)]
    
    return processed_data

def postprocess_best_move(prediction):
    # Преобразование выходных данных модели в читаемый формат
    # Ваши шаги по обратной предобработке данных здесь
    
    # Пример: Возвращаем строку "Ход: {предсказанный ход}"
    return f"Ход: {int(prediction[0])}"

# Tkinter интерфейс
def on_submit():
    opponent_move = entry.get()
    best_move = suggest_best_move(opponent_move)
    messagebox.showinfo("Лучший ход", best_move)

def preprocess_opponent_move(opponent_move):
    # Преобразование входных данных в формат, понимаемый моделью
    # Ваши шаги по предобработке данных хода противника здесь
    
    return processed_data

def postprocess_best_move(prediction):
    # Преобразование выходных данных модели в читаемый формат
    # Ваши шаги по обратной предобработке данных здесь
    
    return best_move

# Tkinter интерфейс
def on_submit():
    opponent_move = entry.get()
    best_move = suggest_best_move(opponent_move)
    messagebox.showinfo("Лучший ход", f"Рекомендуемый ход: {best_move}")

root = Tk()
root.title("Шахматы")

# Выбор цвета
color_label = Label(root, text="Выберите цвет за который играете:")
color_label.pack()

color_var = StringVar(root)
color_var.set("white")  # Установите значение по умолчанию
color_menu = OptionMenu(root, color_var, "white", "black")
color_menu.pack()

# Ввод хода противника
entry_label = Label(root, text="Введите ход противника:")
entry_label.pack()

entry = Entry(root)
entry.pack()

# Кнопка отправки
submit_button = Button(root, text="Подтвердить", command=on_submit)
submit_button.pack()

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "G:\Python Anaconda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Кирилл\AppData\Local\Temp\ipykernel_18228\391318393.py", line 78, in on_submit
    best_move = suggest_best_move(opponent_move)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Кирилл\AppData\Local\Temp\ipykernel_18228\391318393.py", line 32, in suggest_best_move
    opponent_move_data = preprocess_opponent_move(opponent_move)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Кирилл\AppData\Local\Temp\ipykernel_18228\391318393.py", line 67, in preprocess_opponent_move
    return processed_data
           ^^^^^^^^^^^^^^
NameError: name 'processed_data' is not defined
